In [ ]:
# Step 1: Setup the Environment
!pip install transformers datasets evaluate

In [ ]:
# Step 2: Load Dataset
from datasets import load_dataset

# Load the Yelp Review dataset
dataset = load_dataset("yelp_review_full")

# Check out the dataset content
print(dataset["train"][100])

{'label': 0, 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I

In [ ]:
# Step 3: Tokenization
from transformers import AutoTokenizer

# Load the tokenizer for DistilBERT
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Define a tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply it to the whole dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Step 4: Prepare Training and Evaluation Sets
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
# Step 5: Model Setup
from transformers import AutoModelForSequenceClassification

# Load a pre-trained DistilBERT model for classification (5 output labels)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 6: Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="distilbert_trainer",  # Changed from "test_trainer"
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


## Key Steps and Approach

In this notebook, I followed Hugging Face’s recommended approach for fine-tuning transformer models using the `Trainer` class.

- **Data Preprocessing**:  
  I used the `yelp_review_full` dataset from the Hugging Face Datasets library. The text data was tokenized using each model’s specific tokenizer with `padding="max_length"` and `truncation=True` to ensure compatibility with transformer input limits.

- **Model Configuration**:  
  For both models, I used `AutoModelForSequenceClassification` with `num_labels=5` (to match the dataset). I configured the training arguments to:
  - Save the model at each epoch
  - Evaluate after each epoch
  - Run 2–3 epochs with small batch sizes to manage memory on CPU

- **Training & Evaluation**:  
  I trained both models using the `Trainer` API and evaluated their performance with the `compute_metrics()` function. I used accuracy as the evaluation metric.

In [ ]:
# Step 7: Evaluation Metrics
import numpy as np
import evaluate

# Load the accuracy metric
metric = evaluate.load("accuracy")

# Function that calculates the metric from predictions
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Step 8: Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-17-3721faf9f216>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Step 9: Train Model
import os
os.environ["WANDB_DISABLED"] = "true" # Disable wandb logging

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.139451,0.538000
2,No log,1.028519,0.555000
3,No log,0.984627,0.604000


TrainOutput(global_step=375, training_loss=1.116651123046875, metrics={'train_runtime': 11974.9009, 'train_samples_per_second': 0.251, 'train_steps_per_second': 0.031, 'total_flos': 397423457280000.0, 'train_loss': 1.116651123046875, 'epoch': 3.0})

In [ ]:
# Step 10: Evaluate the Model
trainer.evaluate()

{'eval_loss': 0.9846268892288208,
 'eval_accuracy': 0.604,
 'eval_runtime': 808.6032,
 'eval_samples_per_second': 1.237,
 'eval_steps_per_second': 0.155,
 'epoch': 3.0}

## Modifications & Practical Considerations

I originally planned to fine-tune a larger BERT model (`google-bert/bert-base-cased`), but training was too slow and memory-intensive on Colab’s free tier (CPU-only, 12.7 GB RAM). To address this:

- I switched to `distilbert-base-uncased` for faster training while still achieving solid performance (Accuracy: 60.4%)
- I worked with small subsets of 1,000 (and later 500) samples to reduce memory load and speed up training
- I disabled W&B logging and added a memory cleanup cell using `gc.collect()` and `torch.cuda.empty_cache()` to avoid runtime crashes

To further mitigate the risk of disconnections, I ran the cleanup cell before training to free up memory. While originally intended for GPU memory, it helps remove unused objects from memory and proved useful for running both models in a single notebook without restarting.

As a second model I fine-tuned `prajjwal1/bert-tiny`. Although its accuracy was lower (26.4%), it trained very quickly and helped me practice reusing the same pipeline with different model architectures.

In [ ]:
# Memory Cleanup Cell (run before training)
import gc
gc.collect()

import torch
torch.cuda.empty_cache()

In [ ]:
# Step 11: Fine-Tune Second Model – TinyBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load tokenizer and model
tiny_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
tiny_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=5)

# Tokenize again using TinyBERT's tokenizer
def tiny_tokenize_function(examples):
    return tiny_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tiny_tokenized_datasets = dataset.map(tiny_tokenize_function, batched=True)

# Small datasets for fast training
tiny_train_dataset = tiny_tokenized_datasets["train"].shuffle(seed=42).select(range(500))
tiny_eval_dataset = tiny_tokenized_datasets["test"].shuffle(seed=42).select(range(500))

# Training arguments
tiny_training_args = TrainingArguments(
    output_dir="tinybert_trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
tiny_trainer = Trainer(
    model=tiny_model,
    args=tiny_training_args,
    train_dataset=tiny_train_dataset,
    eval_dataset=tiny_eval_dataset,
    tokenizer=tiny_tokenizer,
    compute_metrics=compute_metrics,
)

# Train
os.environ["WANDB_DISABLED"] = "true"
tiny_trainer.train()

# Evaluate
tiny_trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-31-60501a5ef1d1>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  tiny_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.597226,0.254000
2,No log,1.595570,0.264000


{'eval_loss': 1.5955696105957031,
 'eval_accuracy': 0.264,
 'eval_runtime': 9.436,
 'eval_samples_per_second': 52.989,
 'eval_steps_per_second': 6.677,
 'epoch': 2.0}